In [ ]:
# import neccessary package
import numpy as np
import cv2
import matplotlib.pyplot as plt
import glob
import pickle
import os
from moviepy.editor import VideoFileClip
from IPython.display import HTML
import matplotlib.image as mpimg

## 1. Camera Calibration and Image Undistortion

In [ ]:
# function to calibration the vehicle camera
def camera_calibration(image_directory, cal_image):
    objp = np.zeros((6 * 9, 3), np.float32)
    objp[:, :2] = np.mgrid[0:9, 0:6].T.reshape(-1, 2)
    objpoints = []  # 3d points in real world space
    imgpoints = []  # 2d points in image plane.
    images = glob.glob(image_directory)

    # loop over each image in the image_directory
    for idx, fname in enumerate(images):
        img = cv2.imread(fname)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Find the chessboard corners
        ret, corners = cv2.findChessboardCorners(gray, (9, 6), None)

        # If found, add object points, image points
        if ret == True:
            objpoints.append(objp)
            imgpoints.append(corners)

            # Draw and display the corners
            cv2.drawChessboardCorners(img, (9, 6), corners, ret)
            cv2.imshow('img', img)
            cv2.waitKey(500)

    # Do camera calibration on a given test image
    img = cv2.imread(cal_image)
    img_size = (img.shape[1], img.shape[0])
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size, None, None)

    # Save the camera calibration result (mtx, dist) in the dictionary for late use
    dist_pickle = {}
    dist_pickle['mtx'] = mtx
    dist_pickle['dist'] = dist
    pickle.dump(dist_pickle, open('camera_cal.p', 'wb'))
    #plt.show()
    return mtx, dist

In [ ]:
# After calibration the vehicle camera, use this function to undistort images
def undistortion (image, pickle_file, print_flag = None, image_directory = None, cal_image = None):
    # check to see if the pickle_file already existed, then load camera calibration parameters 
    if os.path.isfile(pickle_file):
        dist_pickle = pickle.load(open(pickle_file, 'rb'))
        mtx = dist_pickle['mtx']
        dist = dist_pickle['dist']
    else:
        # if there is no pickle_file, do the camera calibration
        mtx, dist = camera_calibration(image_directory,cal_image)
    img = cv2.imread(image)
    dst = cv2.undistort(img, mtx, dist, None, mtx)
    # write a undistorted image on a file
    cv2.imwrite('output_images/undistored_image.jpg', dst)

    if print_flag == 'yes':
        # Visualize undistortion
        f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))
        ax1.imshow(img, cmap='gray')
        ax1.set_title('Original Image', fontsize=10)
        ax2.imshow(dst, cmap='gray')
        ax2.set_title('Undistorted Image', fontsize=10)
        plt.show()
    return dst


In [ ]:
# camera calibration
# image_directory = 'camera_cal/*.jpg'
# cal_image = 'camera_cal/calibration4.jpg'
# mtx, dist = camera_calibration(image_directory,cal_image)

# Undistort the test image
test_image = 'test_images/test6.jpg'
pickle_file = 'camera_cal.p'
dst = undistortion(test_image,pickle_file, print_flag= 'yes')

## 2. Gradient and Color Threshold
### (a). Function definition

In [ ]:
# Define a function to threshold an image for a given range and Sobel kernel
def dir_threshold(image, sobel_kernel=3, thresh=(0, np.pi / 2)):
    # convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    # use bilateral filter to filter out noises while preserve edges of images
    gray = cv2.bilateralFilter(gray,5,9,9)

    # Calculate the x and y gradients
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)

    # Take the absolute value of the gradient direction,
    # apply a threshold, and create a binary image result
    absgraddir = np.arctan2(np.absolute(sobely), np.absolute(sobelx))
    binary_direction = np.zeros_like(absgraddir)
    binary_direction[(absgraddir >= thresh[0]) & (absgraddir <= thresh[1])] = 1

    # Return the binary image
    return binary_direction


# Define a sobel function that takes an image, gradient orientation,
# and threshold min / max values.
def apply_sobel (image, orient='x', sobel_threshmin = 0, sobel_threshmax = 100, sobel_kernel = 3):
    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # use bilateral filter to filter out noises while preserve edges of images
    gray = cv2.bilateralFilter(gray,5,9,9)

    # Depend on the given orientation, select the right orient
    if orient == 'x':
        sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize= sobel_kernel)
        abs_sobelx = np.absolute(sobelx)
        scaled_sobelx = np.uint8(255 * abs_sobelx / np.max(abs_sobelx))
        scaled_sobel = scaled_sobelx

    elif orient == 'y':
        sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize= sobel_kernel)
        abs_sobely = np.absolute(sobely)
        scaled_sobely = np.uint8(255 * abs_sobely / np.max(abs_sobely))
        scaled_sobel = scaled_sobely

    sobel_binary = np.zeros_like(scaled_sobel)
    sobel_binary[(scaled_sobel >= sobel_threshmin) & (scaled_sobel <= sobel_threshmax)] = 1
    return sobel_binary

# Define a function to return the magnitude of the gradient
# for a given sobel kernel size and threshold values
def mag_thresh(img, sobel_kernel=3, mag_thresh=(0, 255)):
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # use bilateral filter to filter out noises while preserve edges of images
    gray = cv2.bilateralFilter(gray,5,9,9)

    # Take both Sobel x and y gradients
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)

    # Calculate the gradient magnitude
    gradmag = np.sqrt(sobelx**2 + sobely**2)

    # Rescale to 8 bit
    scale_factor = np.max(gradmag)/255
    gradmag = (gradmag/scale_factor).astype(np.uint8)

    # Create a binary image of ones where threshold is met, zeros otherwise
    mag_binary = np.zeros_like(gradmag)
    mag_binary[(gradmag >= mag_thresh[0]) & (gradmag <= mag_thresh[1])] = 1

    return mag_binary

# Define a function to return the color threshold for a given S-channel of HLS color space
def apply_hls_color_threshold (image, hls_threshmin=0, hls_threshmax=100):
    hls = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
    # use bilateral filter to filter out noises while preserve edges of images
    hls = cv2.bilateralFilter(hls,5,9,9)
    h = hls[:,:,0]
    l = hls[:,:,1]
    s = hls[:,:,2]
    hls_binary = np.zeros_like(s)
    # Create a binary image of ones where threshold is met, zeros otherwise
    hls_binary[(s > hls_threshmin) & (s <= hls_threshmax)] = 1
    return  hls_binary

# Define a function to return the color threshold for a given S-channel of HSV color space
def apply_hsv_color_threshold (image, hsv_threshmin=0, hsv_threshmax=100):
    hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    # use bilateral filter to filter out noises while preserve edges of images
    hsv = cv2.bilateralFilter(hsv,5,9,9)
    h = hsv[:,:,0]
    s = hsv[:,:,1]
    v = hsv[:,:,2]
    hsv_binary = np.zeros_like(s)
    # Create a binary image of ones where threshold is met, zeros otherwise
    hsv_binary[(s > hsv_threshmin) & (s <= hsv_threshmax)] = 1
    return hsv_binary

# Define a function to combine different binary images
def combined_threshold (image, print_flag = None):
    # Define the region of interest so that later will be use to  mask out
    # the texture inside the lane.
    img_size = np.shape(image)
    halfTop = np.uint(img_size[0] / 1.5)
    halfBottom = np.uint(img_size[0])
    center = np.uint(img_size[1] / 2)
    TopLeft = np.uint(center - 0.05 * (img_size[1] / 2))
    TopRight = np.uint(center + 0.05 * (img_size[1] / 2))
    BottomLeft = np.uint(center - 0.5 * (img_size[1] / 2))
    BottomRight = np.uint(center + 0.7 * (img_size[1] / 2))
    src = np.float32([[TopLeft, halfTop], [TopRight, halfTop], [BottomRight, halfBottom], [BottomLeft, halfBottom]])
    
    sobelx_binary = apply_sobel(image, 'x', sobel_threshmin=20, sobel_threshmax=100, sobel_kernel=3)
    sobely_binary = apply_sobel(image, 'y', sobel_threshmin=20, sobel_threshmax=100, sobel_kernel=3)
    direction_binary = dir_threshold(image,sobel_kernel=3, thresh=(0.7, 1.3))
    hls_binary = apply_hls_color_threshold(image,hls_threshmin=180, hls_threshmax=255)
    hsv_binary = apply_hsv_color_threshold(image, hsv_threshmin=180, hsv_threshmax=255)
    mag_binary = mag_thresh(image,sobel_kernel=3,mag_thresh=(50,255))

    combined_sobelx_hls_binary = np.zeros_like(sobelx_binary)
    #combined_sobelx_hls_binary[(sobelx_binary == 1) | (hls_binary == 1)] = 1
    combined_sobelx_hls_binary[(sobelx_binary == 1 | ((mag_binary == 1) & (direction_binary == 1))) | hls_binary == 1] = 1

    # mask out texture inside the lane
    cv2.fillPoly(combined_sobelx_hls_binary, np.array([src], np.int32), 0)
    
    # plot 6 different gradient and color threshold technique on a given test image
    if print_flag == 'yes':
        plt.figure('Test Images', figsize=(20,10))
        plt.subplot(2,3,1)
        plt.title('sobel x image')
        plt.imshow(sobelx_binary, cmap='gray')

        plt.subplot(2, 3, 2)
        plt.title('sobel y image')
        plt.imshow(sobely_binary, cmap='gray')

        plt.subplot(2, 3, 3)
        plt.title('sobel direction')
        plt.imshow(direction_binary, cmap='gray')

        plt.subplot(2, 3, 4)
        plt.title('hls threshold image')
        plt.imshow(hls_binary, cmap='gray')

        plt.subplot(2, 3, 5)
        plt.title('hsv threshold image')
        plt.imshow(hsv_binary, cmap='gray')

        plt.subplot(2, 3, 6)
        plt.title('combined sobelx and hls image')
        plt.imshow(combined_sobelx_hls_binary, cmap='gray')
        plt.show()
    return combined_sobelx_hls_binary, sobelx_binary, mag_binary, direction_binary, hls_binary

# Define a function to apply the combined gradient and color threshold technique on eight
# test images in the test_image_directory
def test_image_binaries(test_image_directory, pickle_file):
    list_combined_binary = []
    images = glob.glob(test_image_directory)
    for i, image in enumerate(images):
        dst = undistortion(image,pickle_file, print_flag = None)
        combined_sobelx_hls_binary, sobelx_binary, mag_binary, direction_binary, hls_binary \
        = combined_threshold(dst)
        list_combined_binary.append(combined_sobelx_hls_binary)
        plt.figure('Eight Test Images', figsize=(20,10))
        plt.subplot(2,4, i+1)
        plt.title('Image: test%s.jpg'%str(i+1))
        plt.imshow(combined_sobelx_hls_binary, cmap='gray')
    plt.show()
    return list_combined_binary


### (b). Gradient and Color Threshold Visualization

In [ ]:
# Apply gradient and color threshold on the test image
combined_sobelx_hls_binary = combined_threshold(dst, print_flag='yes')

In [ ]:
# Apply combined gradient and color threshold on different images
test_image_directory = 'test_images/*.jpg'
list_combined_binary = test_image_binaries(test_image_directory,pickle_file)

## 3. Perspective Transform

In [ ]:
# Define a region of interest of lane line to apply perspective transform
def boundRectangle(image):
    # Define a region of interest of lane line to apply perspective transform
    img_size = np.shape(image)
    halfTop = np.uint(img_size[0] / 1.5)
    halfBottom = np.uint(img_size[0])
    center = np.uint(img_size[1] / 2)
    TopLeft = np.uint(center - 0.15 * (img_size[1] / 2))
    TopRight = np.uint(center + 0.25 * (img_size[1] / 2))
    BottomLeft = np.uint(center - 0.9 * (img_size[1] / 2))
    BottomRight = np.uint(center + 1.1 * (img_size[1] / 2))
    cv2.line(image,(TopLeft,halfTop),(TopRight, halfTop),(0,255,0),2)
    cv2.line(image, (TopRight, halfTop), (BottomRight, halfBottom), (0, 255, 0), 6)
    cv2.line(image, (BottomRight, halfBottom), (BottomLeft, halfBottom), (0, 255, 0), 6)
    cv2.line(image, (BottomLeft, halfBottom), (TopLeft, halfTop), (0, 255, 0), 6)

    # Choose the four points for source and destination to apply perspective transform
    src = np.float32([[TopLeft, halfTop], [TopRight, halfTop], [BottomRight, halfBottom], [BottomLeft, halfBottom]])
    dst = np.float32([[0, 0], [img_size[1], 0], [img_size[1], img_size[0]], [0, img_size[0]]])

    return src,dst, image

# Define a perspective transform to warp images in bird-eye view, and plot them
def perspective_transform(image, print_flag = None, figsize = None):
    src, dst, img = boundRectangle(image)
    M = cv2.getPerspectiveTransform(src, dst)
    M_inv = cv2.getPerspectiveTransform(dst, src)
    warped = cv2.warpPerspective(image, M, (image.shape[1], image.shape[0]), flags=cv2.INTER_LINEAR)
    unwarped = cv2.warpPerspective(warped, M_inv, (warped.shape[1], warped.shape[0]), flags=cv2.INTER_LINEAR)

    # Visualize
    if print_flag == 'yes':
        plt.figure(figsize=figsize)
        plt.subplot(1,3,1)
        plt.title('Original binary')
        plt.imshow(img, cmap='gray')

        plt.subplot(1, 3, 2)
        plt.title('bird-eye view')
        plt.imshow(warped, cmap='gray')
        plt.show()
    return warped, M, unwarped, M_inv

# Define a function to plot bird-eye view for a list of images
def bird_eye_view (list_combined_binary, figsize=None):
    for image in list_combined_binary:
        warped = perspective_transform(image,print_flag='yes', figsize=figsize)
        cv2.waitKey(500)

In [ ]:
# Draw a region of interest box and warp the test image for bird-eye view
warped = perspective_transform(dst, print_flag='yes', figsize=(20,40))

Bird-view of the 8 test images in the test_image folder

In [ ]:
bird_eye_view(list_combined_binary, figsize=(8,4))

## 4. Finding Lane
### (a). Calculate Lane Line, Lane Curvature and vehicle position

In [ ]:
# Define a function to calculate lane line
def Advanced_Lane_Line(binary_warped):
    # Assuming you have created a warped binary image called "binary_warped"
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[np.uint(binary_warped.shape[0] // 2):, :], axis=0)
    
    # Create an output image to draw on and  visualize the result
    out_img = (np.dstack((binary_warped, binary_warped, binary_warped)) * 255).astype('uint8')
    
    #Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0] / 2)
    leftx_base = np.argmax(histogram[100:midpoint]) + 100
    rightx_base = np.argmax(histogram[midpoint:-100]) + midpoint

    # Choose the number of sliding windows
    nwindows = 9
    # Set height of windows
    window_height = np.int(binary_warped.shape[0] / nwindows)

    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])

    # Current positions to be updated for each window
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Set the width of the windows +/- margin
    margin = 70
    # Set minimum number of pixels found to recenter window
    minpix = 70

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window + 1) * window_height
        win_y_high = binary_warped.shape[0] - window * window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        
        # Draw the windows on the visualization image
        cv2.rectangle(out_img, (win_xleft_low, win_y_low), (win_xleft_high, win_y_high), (0, 255, 0), 2)
        cv2.rectangle(out_img, (win_xright_low, win_y_low), (win_xright_high, win_y_high), (0, 255, 0), 2)
        
        # Identify the nonzero pixels in x and y within the window
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xleft_low) & (
            nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xright_low) & (
            nonzerox < win_xright_high)).nonzero()[0]
        
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds]
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    # Fit a second order polynomial to each
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)

    # Return a dictionary of relevant variables
    lane_map = {}
    lane_map['left_fit'] = left_fit
    lane_map['right_fit'] = right_fit
    lane_map['nonzerox'] = nonzerox
    lane_map['nonzeroy'] = nonzeroy
    lane_map['out_img'] = out_img
    lane_map['left_lane_inds'] = left_lane_inds
    lane_map['right_lane_inds'] = right_lane_inds
    return lane_map

# Define a function to calculate a lane curvature
def calculate_lane_curve(lane_map):
    # Extract relevant variables from lane_map dictionary
    nonzerox = lane_map['nonzerox']
    nonzeroy = lane_map['nonzeroy']
    left_lane_inds = lane_map['left_lane_inds']
    right_lane_inds = lane_map['right_lane_inds']

    y_eval = 719
    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 15 / 720  # meters per pixel in y dimension
    xm_per_pix = 3.7 / 900  # meters per pixel in x dimension

    # Extract left and right pixel  positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds]
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    # Fit new polynomials to x,y in world space
    left_fit_cr = np.polyfit(lefty * ym_per_pix, leftx * xm_per_pix, 2)
    right_fit_cr = np.polyfit(righty * ym_per_pix, rightx * xm_per_pix, 2)
    # Calculate the new radii of curvature
    left_curverad = ((1 + (2 * left_fit_cr[0] * y_eval * ym_per_pix + left_fit_cr[1]) ** 2) ** 1.5) / np.absolute(
        2 * left_fit_cr[0])
    right_curverad = ((1 + (2 * right_fit_cr[0] * y_eval * ym_per_pix + right_fit_cr[1]) ** 2) ** 1.5) / np.absolute(
        2 * right_fit_cr[0])

    # print('left curve: ', left_curverad, ' ,right curve:  ', right_curverad)
    return left_curverad, right_curverad

# Define a function to calculate lane curvature and vehicle offset
def calculate_vehicle_position(image, lane_map):
    # Calculate vehicle offset from lane center, in meters
    left_fit = lane_map['left_fit']
    right_fit = lane_map['right_fit']
    
    # Calculate vehicle center offset in pixels
    bottom_y = image.shape[0] - 1
    bottom_x_left = left_fit[0] * (bottom_y ** 2) + left_fit[1] * bottom_y + left_fit[2]
    bottom_x_right = right_fit[0] * (bottom_y ** 2) + right_fit[1] * bottom_y + right_fit[2]
    vehicle_position = image.shape[1] / 2 - (bottom_x_left + bottom_x_right) / 2

    # Convert pixel offset to meters
    xm_per_pix = 3.7 / 700  # meters per pixel in x dimension
    vehicle_position *= xm_per_pix
    # print('vehicle position: ', vehicle_position)
    return vehicle_position

# Define a function to draw color map on lane
def drawing(undist, left_fit, right_fit, m_inv, left_curve, right_curve, vehicle_position):
    # Define the y-points to draw color map
    ploty = np.linspace(0, undist.shape[0] - 1, undist.shape[0])
    
    # Fit the left and right lane in second order polynomial curve
    left_fitx = left_fit[0] * ploty ** 2 + left_fit[1] * ploty + left_fit[2]
    right_fitx = right_fit[0] * ploty ** 2 + right_fit[1] * ploty + right_fit[2]
    
    # Define color warp to draw color map on lane lines
    color_warp = np.zeros((720, 1280, 3), dtype='uint8')
    
    # Find the point coordinates on left and right lanes
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))
    
    # Color fill with yellow for coordinate points
    cv2.fillPoly(color_warp, np.int_([pts]), (255, 255, 0))
    
    # Unwarp back to the original images
    newwarp = cv2.warpPerspective(color_warp, m_inv, (undist.shape[1], undist.shape[0]))
    result = cv2.addWeighted(undist, 1, newwarp, 0.3, 0)
    
    
    # Calculate the lane curvature and vehicle offset
    avg_curve = (left_curve + right_curve) / 2
    label_str = 'Radius of curvature: %.1f m' % avg_curve
    result = cv2.putText(result, label_str, (30, 40), 0, 1, (0, 0, 0), 3, cv2.FONT_HERSHEY_DUPLEX)
    label_str = 'Vehicle offset from lane center: %.3f m' % vehicle_position
    result = cv2.putText(result, label_str, (30, 70), 0, 1, (0, 0, 0), 3, cv2.FONT_HERSHEY_DUPLEX)
    plt.imshow(result)
    plt.show()
    return result

### (b) Bird-vew with lane finding drawing

In [ ]:
# Define a function to visualize birdeye view with rectangle boxes
def birdview_rectangle_visualization(binary_warped, lane_map):
    # Extract relevant variables from lane_map dictionary
    left_fit = lane_map['left_fit']
    right_fit = lane_map['right_fit']
    nonzerox = lane_map['nonzerox']
    nonzeroy = lane_map['nonzeroy']
    out_img = lane_map['out_img']
    left_lane_inds = lane_map['left_lane_inds']
    right_lane_inds = lane_map['right_lane_inds']

    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0] - 1, binary_warped.shape[0])
    left_fitx = left_fit[0] * ploty ** 2 + left_fit[1] * ploty + left_fit[2]
    right_fitx = right_fit[0] * ploty ** 2 + right_fit[1] * ploty + right_fit[2]
    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]
    plt.imshow(out_img)
    plt.plot(left_fitx, ploty, color='yellow')
    plt.plot(right_fitx, ploty, color='yellow')
    plt.xlim(0, 1280)
    plt.ylim(720, 0)
    plt.show()

# Define a function to visualize birdeye view with window search area
def birdview_visualization(binary_warped, lane_map):
    left_fit = lane_map['left_fit']
    right_fit = lane_map['right_fit']
    nonzerox = lane_map['nonzerox']
    nonzeroy = lane_map['nonzeroy']
    left_lane_inds = lane_map['left_lane_inds']
    right_lane_inds = lane_map['right_lane_inds']

    # Create an image to draw on and an image to show the selection window
    out_img = np.dstack((binary_warped, binary_warped, binary_warped)) * 255
    window_img = np.zeros_like(out_img)

    # Color in left and right line pixels
    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]

    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0] - 1, binary_warped.shape[0])
    left_fitx = left_fit[0] * ploty ** 2 + left_fit[1] * ploty + left_fit[2]
    right_fitx = right_fit[0] * ploty ** 2 + right_fit[1] * ploty + right_fit[2]

    # Generate a polygon to illustrate the search window area
    # And recast the x and y points into usable format for cv2.fillPoly()
    margin = 70
    left_line_window1 = np.array([np.transpose(np.vstack([left_fitx - margin, ploty]))])
    left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx + margin, ploty])))])
    left_line_pts = np.hstack((left_line_window1, left_line_window2))
    right_line_window1 = np.array([np.transpose(np.vstack([right_fitx - margin, ploty]))])
    right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx + margin, ploty])))])
    right_line_pts = np.hstack((right_line_window1, right_line_window2))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(window_img, np.int_([left_line_pts]), (0, 255, 0))
    cv2.fillPoly(window_img, np.int_([right_line_pts]), (0, 255, 0))
    result = cv2.addWeighted(out_img, 1, window_img, 0.3, 0)
    plt.imshow(result)
    plt.plot(left_fitx, ploty, color='yellow')
    plt.plot(right_fitx, ploty, color='yellow')
    plt.xlim(0, 1280)
    plt.ylim(720, 0)
    plt.show()
    return result

### (c). Process images to find lane line and perform visualization

In [ ]:
# Define a function to annotate the input image with lane line detection and markings
def process_image(img_in):
    #Annotate the input image with lane line markings
    #Returns annotated image

    global mtx, dist

    # Undistort, threshold, perspective transform
    undist = cv2.undistort(img_in, mtx, dist, None, mtx)
    img, abs_bin, mag_bin, dir_bin, hls_bin = combined_threshold(undist)
    binary_warped, binary_unwarped, m, m_inv = perspective_transform(img)

    lane_map = Advanced_Lane_Line(binary_warped)
    left_fit = lane_map['left_fit']
    right_fit = lane_map['right_fit']
    left_curve, right_curve = calculate_lane_curve(lane_map)
    vehicle_position = calculate_vehicle_position(undist, lane_map)

    # Perform final visualization on top of original undistorted image
    # birdview_rectangle_visualization(binary_warped, lane_map)
    # birdeyeview = birdview_visualization(binary_warped, lane_map)
    result = drawing(undist, left_fit, right_fit, m_inv, left_curve, right_curve, vehicle_position)

    return result

In [ ]:
# Define a function to display the lane search window in birdeye-view mode
def process_birdeyeview(img_in):

    global mtx, dist

    # Undistort, threshold, perspective transform
    undist = cv2.undistort(img_in, mtx, dist, None, mtx)
    img, abs_bin, mag_bin, dir_bin, hls_bin = combined_threshold(undist)
    
    # warp the combined gradient/color threshold image
    binary_warped, binary_unwarped, m, m_inv = perspective_transform(img)
    
    # warp the undistortion image
    warped, unwarped, m, m_inv = perspective_transform(undist)

    lane_map = Advanced_Lane_Line(binary_warped)
    left_fit = lane_map['left_fit']
    right_fit = lane_map['right_fit']
    binary_warped = birdview_visualization(binary_warped, lane_map)
    
    return warped, binary_warped, lane_map

In [ ]:
# Define a function to display lane curvature and vehicle offset on image
def text_curvature_offset(img, view, lane_map):
    # Calculate the lane curvature and vehicle offset
    left_curve, right_curve = calculate_lane_curve(lane_map)
    vehicle_position = calculate_vehicle_position(img, lane_map)
    
    # Calculate the average curvature 
    avg_curve = (left_curve + right_curve) / 2
    label_str = 'Radius of curvature: %.1f m' % avg_curve
    view = cv2.putText(view, label_str, (30, 40), 0, 1, (255, 255, 255), 6, cv2.FONT_HERSHEY_DUPLEX)
    label_str = 'Vehicle offset from lane center: %.3f m' % vehicle_position
    view = cv2.putText(view, label_str, (30, 70), 0, 1, (255, 255, 255), 6, cv2.FONT_HERSHEY_DUPLEX)
    plt.imshow(img)
    plt.show()
    return view

In [ ]:
# Define a function to display combined_threshold images
def process_combined_thresh(img_in):
    #Annotate the input image with lane line markings
    #Returns annotated image

    global mtx, dist

    # Undistort, threshold, perspective transform
    undist = cv2.undistort(img_in, mtx, dist, None, mtx)
    img, abs_bin, mag_bin, dir_bin, hls_bin = combined_threshold(undist)
    img_out = (np.dstack((img, img, img)) * 255).astype('uint8')
    return img_out

In [ ]:
# Define a function to display multiview of lane tracking in a full HD.
def multiview(img_in):
    # Define the main view in full HD.
    view = np.zeros((1080, 1920, 3), dtype=np.uint8)
    result = process_image(img_in)
    
    # warp image to birdeye-view
    warped, binary_warped, lane_map = process_birdeyeview(img_in)
    
    # Define the position of the lane tracking on the main view
    view[0:720, 0:1280] = result
    
    # Define the position of the birdeye-view image on the main view
    view[0:330, 1300:1900] = cv2.resize(warped, (600,330),interpolation=cv2.INTER_AREA)
    
    # Define the position of the birdeye-view of the combined gradient/color threshold image on the main
    # view
    view[370:700, 1300:1900] = cv2.resize(binary_warped,(600,330), interpolation=cv2.INTER_AREA)
    
    # Define the position to display curvature and vehicle offset information
    view[750: 1000, 0:800] = text_curvature_offset(binary_warped, view[750: 1000, 0:800], lane_map)
    
    #Define the position to display combined threshold images
    combined_thresh = process_combined_thresh(img_in)
    view[750:998, 840:1280] = cv2.resize(combined_thresh, (440, 248), interpolation=cv2.INTER_AREA)
    
    return view

In [ ]:
dist_pickle = pickle.load(open(pickle_file,'rb'))
mtx = dist_pickle['mtx']
dist = dist_pickle['dist']
img_file = 'test_images/test6.jpg'
img = mpimg.imread(img_file)
result = process_image(img)

In [ ]:
def process_video(input_file, output_file):
    #Given input_file video, save annotated video to output_file
    video = VideoFileClip(input_file)
    #annotated_video = video.fl_image(process_image)
    #annotated_video = video.fl_image(process_birdeyeview)
    annotated_video = video.fl_image(multiview)
    annotated_video.write_videofile(output_file, audio=False)

In [ ]:
process_video('project_video.mp4', 'submit_multiview_with_displayInfo.mp4')